# Setting up an Experiment Baseline Framework

This serves as a baseline framework for the multi-label neural NILM experiments.

### Import Python Modules

In [1]:
from __future__ import print_function, division
import warnings

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')


from data import Environment, get_on_off
from data.generator import Seq2Seq, Seq2Point 

from models import FCN, ResNet, ConvGRU, ConvLSTM, FCN_AE

from experiments import Experiment
from experiments.metrics import validation_report, evaluation_report, confusion_matrix_report, roc_report

from utils.path_finder import NILMTK_SOURCE, SCENARIOS, SOURCES, PRETRAINED, RESULTS

In [2]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

### Setup the Experiment Params

In [6]:
SCENARIO = 4
BATCH_SIZE = 32
WINDOW_SIZE = 100
EPOCHS = 1
LRN_RATE = 0.0001
SPEED = 1

In [ ]:
CLASSIFIERS = {"FCN": FCN(1,3), 
               "ResNet": ResNet(1,3), 
               "ConvGRU": ConvGRU(1,3), 
               "ConvLSTM": ConvLSTM(1,3), 
               "FCN_AE":FCN_AE(1,3)
              }

### Run the Experiment

In [7]:
experiment = Experiment(scenario=SCENARIO)

In [ ]:
for name, clf in CLASSIFIERS.items():

    experiment.setup_running_params(model=clf, epochs=EPOCHS, window=WINDOW_SIZE, batch_size=BATCH_SIZE, lrn_rate=LRN_RATE, speed=SPEED)
    experiment.run(cv=5)

### Evaluate Experiment Outputs

In [ ]:
measures = ['fridge', 'dish washer', 'washer dryer', 'macro']  
metric = "F1 Score" 

target_results = {}
model_results = {}

for name in CLASSIFIERS.keys():
    
    target_filename = os.path.join(RESULTS, "scenario-{}/{}/{}-min-target-results.csv".format(experiment.scenario, name, experiment.window))
    target_results[name] = pd.read_csv(target_filename, index_col=0)

    model_filename = os.path.join(RESULTS, "scenario-{}/{}/{}-min-model-results.csv".format(experiment.scenario, name, experiment.window))
    model_results[name] = pd.read_csv(model_filename, index_col=0)

In [ ]:
scores = {}

for i in measures:
    
    scores[i] = {}
    for name in CLASSIFIERS.keys():
        
        
        if i == "macro":
            scores[i][name] = model_results[name].loc[i, metric]
        else:
            scores[i][name] = target_results[name].loc[i, metric]
            